# Практика
Используемые библиотеки

In [2]:
import pandas as pd
import numpy as np
import requests
import io
import re


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [61]:
"""url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))"""
df=pd.read_csv("him_sad.csv", sep=';')
df.columns

Index(['Показатель', 'Единица измерений', 'Результат анализа', 'Норматив'], dtype='object')

In [67]:
def float_ext(value: str) -> float:
    return float(value.replace(',', '.'))

def cmp(value: float, s: str) -> bool:
    parts = s.split(' ')
    cond = parts[0] + ' ' + parts[1]
    if cond == 'в пределах':
        begin, end = list(map(float_ext, parts[2].split('-')))
        return (begin <= value) and (value <= end)
    elif cond == 'не более':
        return value <= float_ext(parts[2].split('-')[0])

def cmp_save(value: str, cond: str) -> bool:
    try:
        return cmp(float(value), cond)
    except Exception as e:
        return False
df['В пределах нормы'] = df.apply(lambda x: cmp_save(x['Результат анализа'], x['Норматив']), axis=1)
df

,Показатель,Единица измерений,Результат анализа,Норматив,В пределах нормы
0,pH,единицы pH,8.4,в пределах 6-9,True
1,Запах,баллы,1,не более 2-3,True
2,Цветность,градусы,б/цвета,не более 30,False
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,True
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",True
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",True
6,Нитраты (по NO3),мг/дм3,24,не более 45,True
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",True
8,Хлориды (Cl),мг/дм3,200,не более 350,True
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [37]:
box = np.full(4, 'apple')
box = np.append(box, np.full(5, 'orange'))
ok_results = 0
for _ in range(10000):
    box = np.full(4, 'apple')
    box = np.append(box, np.full(5, 'orange'))
    if (np.random.choice(box, 3, replace=False) == 'orange').all():
        ok_results += 1
print(ok_results / 10000.0)
#ДА!


0.1186


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [56]:
two_details = 0
for _ in range(10000):
    details = np.full(7, "standard")
    details = np.append(details, np.full(3, 'nonstandard'))
    np.random.shuffle(details)
    for index, detail in enumerate(details):
        if detail == 'standard':
            if index == 1:
                two_details += 1
            break
print(two_details / 10000.0)
#ДА!

0.2317
